In [ ]:
import csv
import cv2
import numpy as np

lines = []
with open("driving_log.csv") as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)

images = []
measurements = []
counter = 0
for line in lines:
    sourcepath = line[0]
    filename = sourcepath.split('\\')[-1]
    currentpath = "IMG/" + filename
    image = cv2.imread(currentpath)
    images.append(image)
    measurements.append(float(line[3]))
    counter = counter + 1
    
augmented_images, augmented_measurements = [], []
for image, measurement in zip(images, measurements):
    augmented_images.append(image)
    augmented_measurements.append(measurement)
    augmented_images.append(cv2.flip(image, 1))
    augmented_measurements.append(measurement*-1.0)
    
x_train = np.array(images)
y_train = np.array(measurements)

from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D

model = Sequential()
model.add(Lambda(lambda x: (x/255.0), input_shape = (160, 320, 3)))
model.add(Convolution2D(6,5,5,activation="relu"))
model.add(MaxPooling2D())
model.add(Convolution2D(6,5,5,activation="relu"))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(120))
model.add(Dense(84))
model.add(Dense(1))
model.compile(loss='mse', optimizer='adam')
model.fit(x_train, y_train, validation_split=0.2, shuffle = True, epochs = 5)
model.save('model.h5')

scores = model.evaluate(x_train, y_train)
print(scores)

/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launcher.py:40: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(6, (5, 5), activation="relu")`
/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launcher.py:42: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(6, (5, 5), activation="relu")`


Train on 1548 samples, validate on 388 samples
Epoch 1/5
1548/1548 [==============================] - 6s - loss: 1.2175 - val_loss: 0.0215
Epoch 2/5
1548/1548 [==============================] - 6s - loss: 0.0238 - val_loss: 0.0174
Epoch 3/5
1548/1548 [==============================] - 6s - loss: 0.0213 - val_loss: 0.0164